In [1]:
import pandas as pd
import numpy as np


In [4]:
df_loaded = pd.read_csv('./clean_data/clean_data.csv')
df_loaded.drop('Unnamed: 0', axis=1, inplace=True)

In [5]:
df_loaded.head()

,strDrink,strCategory,strGlass,strIngredients,Alc_type,Basic_taste,strInstructions,strMeasures,Value,MeasureName,Value_numeric,Value_ml,Value_gr,Garnish_amount,Garnish_type,MeasureName_copy
0,'57 Chevy with a White License Plate,Cocktail,Highball glass,Creme De Cacao White,Creamy Liqueur,NaN,1. Fill a rocks glass with ice 2.add white cre...,1 oz white,1,oz,1.0,30.0,NaN,NaN,NaN,white
1,1-900-FUK-MEUP,Shot,Old-fashioned glass,Absolut Kurant,Vodka,NaN,Shake ingredients in a mixing tin filled with ...,1/2 oz,1/2,oz,0.5,15.0,NaN,NaN,NaN,oz
2,110 in the shade,Beer,Beer Glass,Lager,Beer,NaN,Drop shooter in glass. Fill with beer,16 oz,16,oz,16.0,480.0,NaN,NaN,NaN,oz
3,151 Florida Bushwacker,Milk / Float / Shake,Beer mug,Malibu Rum,Rum,NaN,Combine all ingredients. Blend until smooth. G...,1/2 oz,1/2,oz,0.5,15.0,NaN,NaN,NaN,oz
4,155 Belmont,Cocktail,White wine glass,Dark Rum,Rum,NaN,Blend with ice. Serve in a wine glass. Garnish...,1 shot,1,shot,1.0,25.0,NaN,NaN,NaN,shot


In [6]:
df = df_loaded.drop('MeasureName_copy', axis=1)

In [11]:
df.drop(df.loc[(df['Value_ml'].isnull()) & (df['Value_gr'].isnull()) & (df['Garnish_amount'].isnull())].index,
                                                                                                           axis=0, inplace=True)

In [21]:
df['Measure'] = np.nan
df['Measure'] = df['Measure'].astype('object')
df['Measure'] = np.where(~df['Garnish_amount'].isnull(), 'garnish', df['Measure'])

In [22]:
df

,strDrink,strCategory,strGlass,strIngredients,Alc_type,Basic_taste,strInstructions,strMeasures,Value,MeasureName,Value_numeric,Value_ml,Value_gr,Garnish_amount,Garnish_type,Measure
0,'57 Chevy with a White License Plate,Cocktail,Highball glass,Creme De Cacao White,Creamy Liqueur,NaN,1. Fill a rocks glass with ice 2.add white cre...,1 oz white,1,oz,1.00,30.0,NaN,NaN,NaN,NaN
1,1-900-FUK-MEUP,Shot,Old-fashioned glass,Absolut Kurant,Vodka,NaN,Shake ingredients in a mixing tin filled with ...,1/2 oz,1/2,oz,0.50,15.0,NaN,NaN,NaN,NaN
2,110 in the shade,Beer,Beer Glass,Lager,Beer,NaN,Drop shooter in glass. Fill with beer,16 oz,16,oz,16.00,480.0,NaN,NaN,NaN,NaN
3,151 Florida Bushwacker,Milk / Float / Shake,Beer mug,Malibu Rum,Rum,NaN,Combine all ingredients. Blend until smooth. G...,1/2 oz,1/2,oz,0.50,15.0,NaN,NaN,NaN,NaN
4,155 Belmont,Cocktail,White wine glass,Dark Rum,Rum,NaN,Blend with ice. Serve in a wine glass. Garnish...,1 shot,1,shot,1.00,25.0,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1776,3-Mile Long Island Iced Tea,Ordinary Drink,Collins Glass,Lemon,NaN,sour,Fill 14oz glass with ice and alcohol. Fill 2/3...,NaN,NaN,oz,NaN,NaN,NaN,1,wedge,garnish
1777,Arizona Twister,Cocktail,Hurricane glass,Pineapple,NaN,sweet,"Just mix in the shots of rum, vodka, and tequi...",1 wedge,1,wedge,NaN,NaN,NaN,1,wedge,garnish
1778,Egg Nog #4,Punch / Party Drink,Punch bowl,Egg White,NaN,egg,In a small mixer bowl beat egg yolks till blen...,6,6,egg,6.00,270.0,NaN,NaN,NaN,NaN
1779,Egg Nog #4,Punch / Party Drink,Punch bowl,Sugar,NaN,sweet,In a small mixer bowl beat egg yolks till blen...,1/4 cup,1/4,cup,0.25,NaN,32.0,NaN,NaN,NaN
